In [1]:
!pip install transformers
!pip install pandas
!pip install datasets
!pip install scikit-learn
!pip install accelerate -U

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from datasets import load_dataset
from transformers import DataCollatorWithPadding
from transformers import Trainer
import numpy as np
import torch
from transformers import TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import RobertaConfig, RobertaForSequenceClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaForSequenceClassification, RobertaTokenizerFast
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

train_data, test_data = load_dataset("Hello-SimpleAI/HC3", name = 'all',  split=['train[:70%]', 'train[70%:]'])

tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_len=512)

Found cached dataset hc3 (/root/.cache/huggingface/datasets/Hello-SimpleAI___hc3/all/1.1.0/5af5910f9f3fe7aace30e32ad4c1ab776ca08183d00e9b2a091308549f69f683)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
test_data.features

{'id': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'human_answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'chatgpt_answers': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'source': Value(dtype='string', id=None)}

In [4]:
train_data[0]

{'id': '0',
 'question': 'Why is every book I hear about a " NY Times # 1 Best Seller " ? ELI5 : Why is every book I hear about a " NY Times # 1 Best Seller " ? Should n\'t there only be one " # 1 " best seller ? Please explain like I\'m five.',
 'human_answers': ['Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you \'re still an " oscar - winning " film . Same thing for best sellers . Also , IIRC the rankings change every week or something like that . Some you might not be best seller one week , but you may be the next week . I guess even if you do n\'t stay there for long , you still achieved the status . Hence , # 1 best seller .',
  "If you 're hearing about it , it 's because it was a very good or very well - publicized book ( or both ) , and almost every g

In [5]:
print(len(train_data), len(test_data))

17025 7297


In [6]:
def prepare_dataset(dataset):
    data = []
    for item in dataset:
        for ans in item['human_answers']:
            data.append({'text': ans, 'label': 'human'})
        for ans in item['chatgpt_answers']:
            data.append({'text': ans, 'label': 'chatgpt'})
    return pd.DataFrame(data)

train_df = prepare_dataset(train_data)
test_df = prepare_dataset(test_data)

In [7]:
train_df

,text,label
0,"Basically there are many categories of "" Best ...",human
1,"If you 're hearing about it , it 's because it...",human
2,"One reason is lots of catagories . However , h...",human
3,There are many different best seller lists tha...,chatgpt
4,salt is good for not dying in car crashes and ...,human
...,...,...
67643,"On Reddit, ""meta"" refers to content that is ab...",chatgpt
67644,"Sure , religion accounts for the opposition to...",human
67645,"The Bible also forbids theft , but it 's okay ...",human
67646,"If separation of church and state meant "" nobo...",human


In [8]:
# create a label mapping 
label2id = {'human':0, 'chatgpt':1}
id2label = {v: k for k, v in label2id.items()}

def encode_examples(df):
    encodings = tokenizer(df['text'].tolist(), truncation=True, padding='longest', max_length=512, return_tensors='pt')
    labels = df['label'].map(label2id).tolist()
    return {**encodings, 'labels': labels}

train_encodings = encode_examples(train_df)
test_encodings = encode_examples(test_df)

In [9]:
# create dataset class 
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != 'labels'}
        item['labels'] = torch.tensor(self.encodings['labels'][idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])


train_dataset = MyDataset(train_encodings)
test_dataset = MyDataset(test_encodings)

In [10]:
# setup Trainer for training
training_args = TrainingArguments(
    output_dir='./results',          
    per_device_train_batch_size=64,
    num_train_epochs=3,  # You might want to increase this
)

# Specify the configuration
config = RobertaConfig.from_pretrained('roberta-base')
config.num_labels = len(label2id)  # Number of classes for classification
config.gradient_checkpointing = True  # Enable gradient checkpointing

# Create a new RoBERTa model for sequence classification
model = RobertaForSequenceClassification(config)

In [11]:
# setup Trainer for evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# training_args = TrainingArguments(
#     output_dir='./results',          
#     per_device_train_batch_size=64,
#     num_train_epochs=3,  # You might want to increase this
# )

trainer = Trainer(
    model=model,                 
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,   
    eval_dataset=test_dataset,          
    data_collator=DataCollatorWithPadding(tokenizer), 
    tokenizer=tokenizer,  
)

# Train the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_710/871423802.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key != 'labels'}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but al

Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# evaluate the model
eval_result = trainer.evaluate()
print(eval_result)